In [ ]:
import pyro
import torch
import gpytorch
import scanpy as sc

adata = sc.read_h5ad("/home/jhaberbe/Projects/spatial-indian-buffet-process/data/16APR2025.h5ad")
adata = adata[adata.obs["cell_type"].eq("Astrocyte") & adata.obs["folder"].eq("99-15")]

/home/jhaberbe/Projects/spatial-indian-buffet-process/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jhaberbe/Projects/spatial-indian-buffet-process/.venv/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [2]:
import torch
import numpy as np
import pandas as pd

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print("CUDA Enabled")

def setup_torch_data(adata, specimen_name: str = "folder"):
    X = torch.tensor(adata[:, (adata.X > 0).mean(axis=0) > 0.05].layers["transcript"])
    size_factor = torch.tensor(np.log((adata.X.sum(axis=1) / adata.X.sum(axis=1).mean())))
    folder = torch.tensor(pd.Categorical(adata.obs[specimen_name]).codes).float()
    return X, size_factor, folder

X, size_factor, folder = setup_torch_data(adata, specimen_name="folder")

X = X.to(device)
size_factor = size_factor.to(device)
folder = folder.to(device)
locs = torch.tensor(adata.obs[["x_centroid", "y_centroid"]].values).to(device)
group_assignments = torch.zeros(X.shape[0])

In [3]:
import sys
sys.path.append("..")

In [4]:
import importlib

import src.model
importlib.reload(src.model)

<module 'src.model' from '/home/jhaberbe/Projects/spatial-indian-buffet-process/notebook/../src/model.py'>

In [5]:
sibp = src.model.SpatialIndianBuffetProcess(adata=adata[:, (adata.X > 0).mean(axis=0) > 0.05], length_scale=400)

In [ ]:
sibp.fit(
    coordinates = locs,
    count_matrix = X,
    group_assignments = folder,
    num_steps = 60_000,
    wandb_kwargs = {
        "project": "sibp",
        "name": "kanna-chameleon"
    }
)

[10300] ELBO loss: 830158.88


In [ ]:
latent_features = sibp.return_latent_features()
return_latent_feature_weights = sibp.return_latent_feature_weights()

NameError: name 'self' is not defined